## This notebook trains the "baseline" classifier of the zebrafish embryo mutants. Baseline classifier is a shallow convolutional classifier which serves as a comparisson for the transfer learning classifier performance. The model contains 4 convolutional layers intermitted by a maxpooling layer. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm 
import tensorflow as tf
import shutil
import os
import tensorflow_datasets as tfds   
from skimage.transform import resize
from skimage.draw import line_nd
from sklearn import metrics
import pandas as pd
import glob
plt.rcParams["axes.grid"] = False
from imageio import imread
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from pathlib import Path

### Loading the zebrafish data

In [ ]:
path_train = Path("training")
path_val = Path("validation")

train_datagen_augmented = ImageDataGenerator(rescale = 1.0/255., brightness_range = (0.7, 1.3), rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True)
val_datagen = ImageDataGenerator( rescale = 1.0/255. ) #no augmentation

data_train = train_datagen_augmented.flow_from_directory(path_train, batch_size = 16, class_mode = 'sparse', target_size = (450, 900))
data_test = val_datagen.flow_from_directory(path_val,  batch_size = 16, class_mode = 'sparse', target_size = (450, 900))

x,y = data_train.next()

In [ ]:
data_train.class_indices

### Building and compiling the model

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32,3,activation="relu", padding="same", input_shape = x.shape[1:]))
model.add(tf.keras.layers.Conv2D(32,3,activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D((4,4)))
model.add(tf.keras.layers.Conv2D(32,3,activation="relu", padding="same"))
model.add(tf.keras.layers.Conv2D(32,3,activation="relu", padding="same"))
model.add(tf.keras.layers.GlobalAveragePooling2D())
#another conv2D x 2 with 32
#another max pol
#global average pooling. kill flatt
model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Flatten()) 
model.add(tf.keras.layers.Dense(4,)) #from_logits = True so no need for softmax here

model.summary()
epochs = 200

rate = 0.001

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=rate),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

#preparing the weights for balanced training
counts = Counter(data_train.classes)
counts_total = sum(counts.values())
class_weights = dict((k, counts_total/v) for k,v in counts.items())

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,patience=2, min_lr=0.00025, verbose = 1)

checkpoint_folder = Path('checkpoints')
checkpoint_filepath = checkpoint_folder/"baseline_classifier"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

### Training the model

In [ ]:
history = model.fit(data_train,
                epochs=epochs,
                validation_data = data_test,
                class_weight=class_weights, callbacks=[model_checkpoint_callback])
np.save(checkpoint_folder/f"baseline_classifier_lr_{rate:.4f}_epochs_{epochs}.npy",history.history)

### Loading best weights

In [ ]:
model.load_weights(checkpoint_filepath)

### Calculating the confusion matrix

In [ ]:
import seaborn as sn #importing here since it affects the plotting of the masks when imported
data_test = val_datagen.flow_from_directory(path_val,  batch_size = 16, class_mode = 'sparse', target_size = (450, 900))

true = list() #list of true labels
predicted = list() #list of predicted labels

for i in range(len(data_test)):
    x, y = data_test.next()
    for j in range(len(x)):
        y_pred = np.argmax(model.predict(x[j:j+1]), axis=-1)
        
        y_true = y[j]
        
        true.append(y_true)
        predicted.append(y_pred)
        
classes = []
for cl in data_test.class_indices:
    classes.append(cl)
    
def calculate_confusion_matrix(classes, true, predicted):
    matrix = metrics.confusion_matrix(true, predicted) #rows - true, columns - predicted
    matrix = matrix.astype(float)


    for i in range(len(matrix)): #scaling per row (per true label)
        matrix[:][i] = matrix[:][i] / sum(matrix[:][i])
    

    df_cm = pd.DataFrame(matrix, index=[classes[0], classes[1], classes[2], classes[3]], columns=[classes[0], classes[1], classes[2], classes[3]])
    # plt.figure(figsize=(10,7))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='1.3f')# font size
    plt.ylabel("True class")
    plt.xlabel("Predicted class") 
    plt.show()
    accuracy = sum(np.diag(matrix))/sum(sum(matrix))
    print(accuracy) 
    return accuracy

accuracy = calculate_confusion_matrix(classes, true, predicted)